#### 작업1유형

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./boston.csv')
df = data.copy()
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,NaN,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,NaN,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,NaN,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,NaN,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,NaN,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


주어진 Dataset에서 CRIM값이 가장 큰 10개의 지역을 구하고
10개의 지역의 CRIM값을 그 중 가장 작은 값으로 대체하라. 그리고
AGE 컬럼 값이 80이상인 대체 된 CRIM 평균값을 구하라

In [3]:
temp = df.sort_values('CRIM', ascending= False)
temp.iloc[:10,0] = temp.iloc[9,0]
temp2 = temp[temp['AGE']>=80]
answer = temp2['CRIM'].mean()
print(answer)

5.759386624999999


In [4]:
data = pd.read_csv('./housing.csv')
df = data.copy()
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


주어진 데이터의 첫번째 행 부터 순서대로 80% 까지의 데이터를 추출 후 'total_bedrooms' 변수의 결측값(NA)을 'total_bedrooms' 변수의 중앙값으로 대체하고,
대체 전의 'total_bedrooms' 변수 표준편차값과 대체 후의 'total_bedrooms' 변수 표준편차 값을 산출하여
대체 전 표준편차 값 - 대체 후 표준편차 값을 구해 출력한다.

In [5]:
df.shape
temp = 20640*0.8
print(temp)

16512.0


In [6]:
bedrooms = df['total_bedrooms'][:16512]
before = bedrooms.std()
median = bedrooms.median()
bedrooms = bedrooms.fillna(median)
after = bedrooms.std()
answer = before - after
print(answer)

1.975147291645726


하우징 데이터에서 'latitude' 컬럼의 이상치를 찾아 이상치들의 합을 산출하시오.
이상치 기준 : 평균 - (표준편차 * 1.5), 평균 + (표준편차 * 1.5)

In [7]:
latitude = df['latitude']
mean = latitude.mean()
std = latitude.std()
lower = mean - std*1.5
upper = mean + std*1.5

outlier = latitude[(lower>latitude) | (upper<latitude)].sum()
print(outlier)

45815.75


#### 작업2유형

데이터 설명 : e-commerce 배송의 정시 도착여부 (1: 정시배송 0 : 정시미배송)

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import r2_score, roc_auc_score, f1_score

In [33]:
X_train = pd.read_csv('./ship/X_train.csv')
y_train = pd.read_csv('./ship/y_train.csv')
X_test = pd.read_csv('./ship/X_test.csv')

In [34]:
X_train = X_train.drop(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'], axis=1)
X_test = X_test.drop(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'], axis=1)
X_train

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
0,6045,4,3,266,5,5,1590
1,44,3,1,174,2,44,1556
2,7940,4,1,154,10,10,5674
3,1596,4,3,158,3,27,1207
4,4395,5,3,175,3,7,4833
...,...,...,...,...,...,...,...
6593,8610,5,2,221,6,4,1952
6594,3390,4,5,256,3,10,4504
6595,5150,3,1,217,4,1,5761
6596,3950,4,5,174,3,8,5576


In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [36]:
X_train_id = X_train.pop('ID')
X_test_id = X_test.pop('ID')

In [37]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(X_train,y_train['Reached.on.Time_Y.N'],test_size=0.2,random_state=2021)

In [38]:
from sklearn.metrics import roc_auc_score

In [39]:
model = LogisticRegression()
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

ValueError: could not convert string to float: '$7'

In [40]:
model = KNeighborsClassifier()
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

ValueError: could not convert string to float: '$7'

In [41]:
model = DecisionTreeClassifier()
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)
roc_auc_score(y_val, pred[:,1])

ValueError: could not convert string to float: '$7'